In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import logging

# Use the webdriver to scrape the target website
- to search the jobs based on categories.
- focus on IT and related jobs first, get all the data if complete IT jobs scraping.

## Open the browser and query the database

In [2]:
logger = logging.getLogger(__name__)
TARGET_WEBSITE = "https://www.104.com.tw/jobs/main/"

driver = webdriver.Chrome(ChromeDriverManager().install()) # open chrome
driver.get(TARGET_WEBSITE) # open the target website
driver.fullscreen_window() # fullscreen to locate the element

def search_IT_jobs():
  """
  The function to search the IT jobs
  """
  sleep(5) # to wait for the elements loading complete
  try:
    btn_cancel = driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div[3]/div/div/button[1]')
    btn_cancel.click()
  except:
    print("no cancel button")
  sleep(1)
  # open the category filter
  category = driver.find_element_by_id('ijob')
  category.click()
  sleep(1)
  # open the IT category
  categories = driver.find_elements_by_class_name('category-item')
  for category in categories:
    if category.text == '資訊軟體系統類':
      category.click()
      break
  sleep(1)
  btn_all = driver.find_element_by_xpath('/html/body/div[12]/div/div[2]/div/div[2]/div[2]/li/a/span[1]/input')
  btn_all.click()
  # confirm
  sleep(1)
  btn_confirm = driver.find_element_by_xpath('/html/body/div[12]/div/div[2]/div/div[3]/button')
  btn_confirm.click()
  sleep(1)
  # search
  btn_search = driver.find_element_by_xpath('/html/body/article[1]/div/div/div[4]/div/button')
  btn_search.click()
search_IT_jobs()




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\alex0\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


## Get the desired data according to the query
- get the job title, company name, location, salary, and job description.
- from 104:
  - first line: job title
  - second line: company name
  - third line: location
  - fourth line: experience
  - fifth line: degree
  - sixth line: job description
  - seventh line: job salary

### Save the data as an object into a list
- each page has 20-30 jobs, so we need to loop through the pages to get all the data.

In [16]:

page = 1
def crawl_one_page_info(job_info):
  """
  Crawl one page of the job information. There could a bug since the
  human power bank advertise certain position without format.
  """
  global page # for counting the page
  sleep(1)
  try:
    job_items = driver.find_elements_by_tag_name('article')
    num_jobs = len(job_items)
    print(f"Page {page} with {num_jobs} jobs.")
    page += 1
  except Exception as e:
    logger.error(e)
  
  for job_item in job_items:
    try:
      job_item_name = job_item.get_attribute('data-job-name')
    except Exception as e:
      logger.error(f"Failed to get job name: {e}")
      job_item_name = 'None'
    try:
      job_item_comapny = job_item.get_attribute('data-cust-name')
    except Exception as e:
      logger.error(f"Failed to get job company: {e}")
      job_item_comapny = 'None'
    try:
      job_item_area = job_item.find_element_by_class_name('job-list-intro').find_elements_by_tag_name('li')[0].text
    except Exception as e:
      logger.error(f"Failed to get job area: {e}")
      job_item_area = 'None'
    try:
      job_item_experience = job_item.find_element_by_class_name('job-list-intro').find_elements_by_tag_name('li')[1].text
    except Exception as e:
      logger.error(f"Failed to get job experience: {e}")
      job_item_experience = 'None'
    try:
      job_item_degree = job_item.find_element_by_class_name('job-list-intro').find_elements_by_tag_name('li')[2].text
    except Exception as e:
      logger.error(f"Failed to get job degree: {e}")
      job_item_degree = 'None'
    try:
      job_item_salary = job_items[0].find_element_by_class_name('job-list-tag').find_elements_by_tag_name('span')[0].text
    except Exception as e:
      logger.error(f"Failed to get job salary: {e}")
      job_item_salary = 'None'
    try:
      job_item_description = job_item.find_elements_by_class_name('job-list-item__info')[0].text
    except Exception as e:
      logger.error(f"Failed to get job description: {e}")
      job_item_description = 'None'

    try:
      # sleep(0.5)
      job_info.append({
        'job_item_name': job_item_name,
        'job_item_comapny': job_item_comapny,
        'job_item_area': job_item_area,
        'job_item_experience': job_item_experience,
        'job_item_degree': job_item_degree,
        'job_item_salary': job_item_salary,
        'job_item_description': job_item_description
      })
    except:
      print("saving process failed")
  return job_info
  

### Crawl all the pages

In [17]:
def crawl_all_page_info():
  """
  The function to crawl all the pages
  """
  page = 1 # reset the page start
  job_info = []
  page_num = len(driver.find_elements_by_xpath('//*[@id="js-job-header"]/div[1]/label[1]/select')[0].find_elements_by_tag_name('option'))
  for i in range(page_num):
    # 第 n/150頁
    sleep(1)
    page_selector = driver.find_elements_by_xpath('//*[@id="js-job-header"]/div[1]/label[1]/select')[0]
    page_selector.click()
    sleep(0.5)
    # 選擇第 n/150頁
    page = page_selector.find_elements_by_tag_name('option')[i]
    sleep(0.5)
    page.click()
    sleep(2) # to wait for the elements loading complete
    crawl_one_page_info(job_info)
  return job_info

# Data processing
- convert the data into a dataframe
- save the dataframe as a csv file

## Data scraping

In [18]:
all_jobs_info = crawl_all_page_info()

Page 1 with 32 jobs.


Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job description: list index out of range
Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"m

Page 2 with 22 jobs.


Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job description: list index out of range
Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"m

Page 3 with 22 jobs.


...

Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job description: list index out of range
Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"m

Page 149 with 22 jobs.


Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job description: list index out of range
Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"m

Page 150 with 22 jobs.


Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job description: list index out of range
Failed to get job area: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job experience: Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-list-intro"}
  (Session info: chrome=95.0.4638.69)

Failed to get job degree: Message: no such element: Unable to locate element: {"m

In [19]:
print(len(all_jobs_info))

3310


## Data processing and saving

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame(all_jobs_info)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3310 entries, 0 to 3309
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   job_item_name         3010 non-null   object
 1   job_item_comapny      3010 non-null   object
 2   job_item_area         3310 non-null   object
 3   job_item_experience   3310 non-null   object
 4   job_item_degree       3310 non-null   object
 5   job_item_salary       3310 non-null   object
 6   job_item_description  3310 non-null   object
dtypes: object(7)
memory usage: 181.1+ KB


In [22]:
df.to_csv('../data/104_IT_jobs.csv', index=False)